# SPARQL with Blazegraph

## Preparation

### Download
* [Blazegraph](https://github.com/blazegraph/database/releases/tag/BLAZEGRAPH_2_1_6_RC)
* [Sample Dataset](https://web.archive.org/web/20190821204658/http://unlocode.rkbexplorer.com/models/dump.tgz)

### Starting the Server

The Blazegraph server can be started with the following command

`!java -server -DentityExpansionLimit=2500000 -Xmx4g -jar ~/Downloads/blazegraph.jar`

> Adjust the path to the blazegraph jar according to the location on your file system. Here the jar is in the Downloads folder `~/Downloads/blazegraph.jar`

Run the command either in
* your terminal (without leading '!'), or
* open a new jupyter notebook and execute the command there (with the leading "!")
  * Stop the server here by interupting the kernal
    
The Server should now be avaliable at `http://172.22.0.1:9999/blazegraph/`.

The SPARQL endpoint of the server is `http://172.22.0.1:9999/blazegraph/sparql`.

## First Steps
### Install python modules

In [ ]:
import sys
def installModule(projectName:str, moduleName:str=None):
    '''Installs and loads the given module if not already installed'''
    if moduleName is None:
        moduleName=projectName
    !python -m pip install -U --no-input $projectName
    print(f'{projectName} installed')
    %reload_ext $moduleName
installModule("pylodstorage", "lodstorage")
installModule("tabulate", "tabulate")

### Setting up the Endpoint

In [ ]:
from lodstorage.sparql import SPARQL
endpointUrl="http://172.22.0.1:9999/blazegraph/sparql"
endpoint=SPARQL(endpointUrl)

#### First Query
Since we did not add any data to the database. We can write a SPARQL query asking for every triple and expect that the result is empty.

In [ ]:
query="""
SELECT ?s ?p ?o
WHERE{
	?s ?o ?p.
}
"""
qres=endpoint.queryAsListOfDicts(query)
print(qres)
#assert( qres == [] )

### Adding a Triple
Inserting or deleting a Triple does not return a result in the form of a records thus the default query function can not be used since it performs a result converson that is not possible on inserts or deletions.

Thus, we use the `rawQuery()` function.

Now add the information that a conference corpus event entity is known as "Event". (Use `rdfs:label`)

In [ ]:
query="""
PREFIX cc: <https://conferencecorpus.bitplan.com/schema#>
INSERT { cc:event rdfs:label "Event"@en } WHERE {}
"""
qres=endpoint.rawQuery(query)
print(qres)
assert(qres.response.code == 200)

In [ ]:
query="""
SELECT ?s
WHERE{
	?s rdfs:label "Event"@en.
}
"""
qres=endpoint.queryAsListOfDicts(query)
print(qres)

#### Adding more triples

Use the following subset of the ConferenceCorpus API request (`https://conferencecorpus.bitplan.com/eventseries/WEBIST`) and transform the result into triples.
1) Try to identify the entites you want to create
2) Define needed properties
3) Build the triples by using the properties to assign either literals or entites to the entites

> You can make use of the following prefixes
>```turtle
>@prefix confref: <http://portal.confref.org/list/>.
>@prefix dblp: <http://dblp.org/db/conf/>.
>@prefix wikicfp: <http://wikicfp.com/>.
>@prefix cc: <https://conferencecorpus.bitplan.com/>.
>@prefix wd: <http://www.wikidata.org/entity/>.
>@prefix schema: <http://schema.org/>.
>@prefix xsd: <http://www.w3.org/2001/XMLSchema#>.
>@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>.
>```


##### Data (confref result for WEBIST 2019)
see webist series in openresearach https://confident.dbis.rwth-aachen.de/or/index.php?title=WEBIST
```json
{
    "acronym": "WEBIST",
    "city": "Vienna",
    "cityWikidataid": "Q1741",
    "country": "Austria",
    "countryIso": "AT",
    "countryWikidataid": "Q40",
    "dblpSeriesId": "conf/webist",
    "endDate": "2019-09-20",
    "eventId": "webist2019",
    "region": "Vienna",
    "regionIso": "AT-9",
    "regionWikidataid": "Q1741",
    "seriesId": "webist",
    "seriesTitle": "Web Information Systems and Technologies",
    "source": "confref",
    "startDate": "2019-09-18",
    "submissionExtended": 0,
    "title": "Web Information Systems and Technologies",
    "url": "http://portal.confref.org/list/webist2019",
    "year": 2019
}
```



#### Possible Solution
Turtle file that could be directly imported to Blazegraph (or Jena/ other triple stores):
```turtle
@prefix confref: <http://portal.confref.org/list/>.
@prefix dblp: <http://dblp.org/db/conf/>.
@prefix wikicfp: <http://wikicfp.com/>.
@prefix cc: <https://conferencecorpus.bitplan.com/>.
@prefix wd: <http://www.wikidata.org/entity/>.
@prefix schema: <http://schema.org/>.
@prefix xsd: <http://www.w3.org/2001/XMLSchema#>.
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>.

confref:webist2019 a cc:Event;
    cc:eventId "webist2019"@en;
    cc:acronym "WEBIST"@en;
    cc:country wd:Q40;
    cc:region wd:Q1741;
    cc:city wd:Q1741;
    cc:endDate "2019-09-20"^^xsd:date;
    cc:startDate "2019-09-18"^^xsd:date;
    cc:title "Web Information Systems and Technologies"@en;
    cc:inEventSeries confref:webist;
    cc:year "2019"^^xsd:int .

confref:webist a cc:EventSeries;
    cc:title "Web Information Systems and Technologies"@en;
    schema:sameAs dblp:webist .

dblp:conf_webist a cc:EventSeries .
    
wd:Q40 a wd:country ;
    rdfs:label "Austria"@en;
    rdfs:label "Republic of Austria"@en;
    rdfs:label "Österreich"@de;
    # Key of the country in openresearch
    cc:iso "AT" .

wd:Q1741 a wd:city;
    a wd:region;
    rdfs:label "Vienna"@en;
    cc:iso "AT-9";
    cc:country wd:Q40 .

```
> Note: The relation between `confref:webist` and `dblp:webist` is here realized with [`schema:sameAs`](https://schema.org/sameAs) and should only be set if both entites reference truly the same entity.
In this example it was evaluated manually but if done programatically you should introduce a new property which contains the semantic information that it is probably the same entity. 

In [ ]:
query="""
PREFIX confref: <http://portal.confref.org/list/>
PREFIX dblp: <http://dblp.org/db/conf/>
PREFIX wikicfp: <http://wikicfp.com/>
PREFIX cc: <https://conferencecorpus.bitplan.com/>
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX schema: <http://schema.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

INSERT { 
confref:webist2019 a cc:Event;
    cc:eventId "webist2019"@en;
    cc:acronym "WEBIST"@en;
    cc:country wd:Q40;
    cc:region wd:Q1741;
    cc:city wd:Q1741;
    cc:endDate "2019-09-20"^^xsd:date;
    cc:startDate "2019-09-18"^^xsd:date;
    cc:title "Web Information Systems and Technologies"@en;
    cc:inEventSeries confref:webist;
    cc:year "2019"^^xsd:int .

confref:webist a cc:EventSeries;
    cc:title "Web Information Systems and Technologies"@en;
    schema:sameAs dblp:webist .

dblp:webist a cc:EventSeries .
    

wd:Q40 a wd:country ;
    rdfs:label "Austria"@en;
    cc:iso "AT" .

wd:Q1741 a wd:city;
    a cc:region;
    rdfs:label "Vienna"@en;
    cc:iso "AT-9";
    cc:country wd:Q40 .

} WHERE {}
"""
qres=endpoint.rawQuery(query)
print(qres)
assert(qres.response.code == 200)

#### Query the data
Write a query to get the event acronym and the title of the series form events that took place in Vienna.

In [ ]:
query="""
PREFIX confref: <http://portal.confref.org/list/>
PREFIX dblp: <http://dblp.org/db/conf/>
PREFIX wikicfp: <http://wikicfp.com/>
PREFIX cc: <https://conferencecorpus.bitplan.com/>
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX schema: <http://schema.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?acronym ?seriesTitle
WHERE{
	?city rdfs:label "Vienna"@en.
    ?event cc:city ?city.
    ?event cc:acronym ?acronym.
    ?event cc:inEventSeries ?series.
    ?series cc:title ?seriesTitle.
}
"""
qres=endpoint.queryAsListOfDicts(query)
print(qres)

### More Complex Queries
Now we add a new event entity for the webist 2019 event from dblp.

In [ ]:
query="""
PREFIX confref: <http://portal.confref.org/list/>
PREFIX dblp: <http://dblp.org/db/conf/>
PREFIX wikicfp: <http://wikicfp.com/>
PREFIX cc: <https://conferencecorpus.bitplan.com/>
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX schema: <http://schema.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

INSERT { 
dblp:webist_2019 a cc:Event;
    cc:inEventSeries dblp:webist;
    cc:title "Proceedings of the 15th International Conference on Web Information Systems and Technologies, WEBIST 2019, Vienna, Austria, September 18-20, 2019."@en;
    # signature elements derived from the title by parsing NLP
    cc:ordinal "15"^^xsd:int;
    cc:startDate "2019-09-18"^^xsd:date;
    cc:endDate "2019-09-20"^^xsd:date;
    cc:month "9"^^xsd:int;
    cc:year "2019"^^xsd:int .

} WHERE {}
"""
qres=endpoint.rawQuery(query)
assert(qres.response.code == 200)

Since we already added the connection with the `schema:sameAs` relation between the series entity from confref and dblp we can now utalize this connection in a query.
The dblp entity has currently no location information but an additional title.

Same query as before (get the event acronym and the title of the series form events that took place in Vienna) but in addition get all titles the event is known by. (Utilize the relation over the series)

In [ ]:
query="""
PREFIX confref: <http://portal.confref.org/list/>
PREFIX dblp: <http://dblp.org/db/conf/>
PREFIX wikicfp: <http://wikicfp.com/>
PREFIX cc: <https://conferencecorpus.bitplan.com/>
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX schema: <http://schema.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?acronym ?title ?year
WHERE{
	?city rdfs:label "Vienna"@en.
    ?event cc:city ?city.
    ?event cc:acronym ?acronym.
    ?event cc:inEventSeries ?series.
    ?event cc:year ?year.
    ?series cc:title ?seriesTitle.
    
    OPTIONAL{
        ?series (schema:sameAs|^schema:sameAs)+ ?sameAsSeries .
        ?sameAsEvent cc:inEventSeries ?sameAsSeries;
            cc:year ?yearB;
            cc:title ?title.
        FILTER(?year = ?yearB)
    }
    
}
"""
qres=endpoint.queryAsListOfDicts(query)
print(qres)

> Alternative: Group and aggregate the different titles of an event with [GROUP_CONCAT](https://www.w3.org/TR/sparql11-query/#defn_aggGroupConcat) and GROUP BY

In [ ]:

query="""
PREFIX confref: <http://portal.confref.org/list/>
PREFIX dblp: <http://dblp.org/db/conf/>
PREFIX wikicfp: <http://wikicfp.com/>
PREFIX cc: <https://conferencecorpus.bitplan.com/>
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX schema: <http://schema.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?acronym (GROUP_CONCAT(?title; SEPARATOR=" | ") AS ?p) ?year
WHERE{
	?city rdfs:label "Vienna"@en.
    ?event cc:city ?city.
    ?event cc:acronym ?acronym.
    ?event cc:inEventSeries ?series.
    ?event cc:year ?year.
    ?series cc:title ?seriesTitle.
    
    OPTIONAL{
        ?series (schema:sameAs|^schema:sameAs)+ ?sameAsSeries .
        ?sameAsEvent cc:inEventSeries ?sameAsSeries;
            cc:year ?yearB;
            cc:title ?title.
        FILTER(?year = ?yearB)
    }
    
}
GROUP BY ?acronym ?year
"""
qres=endpoint.queryAsListOfDicts(query)
print(qres)

### Importing the sample datasets
Go to the [web UI of blazegraph](http://172.22.0.1:9999/blazegraph/#update) and import the files ony by one.
* Browse → Update

#### Test if the import worked by running the following query

In [ ]:
from tabulate import tabulate
query="""
PREFIX unlocode:<http://unlocode.rkbexplorer.com/id/>
PREFIX portal: <http://www.aktors.org/ontology/portal#>
PREFIX support: <http://www.aktors.org/ontology/support#>

SELECT ?townname ?lat ?lon ?regionname ?countryname
WHERE {
  ?town a portal:Town.
  ?town support:has-pretty-name ?townname.
  ?town portal:has-latitude ?lat.
  ?town portal:has-longitude ?lon.
  ?town portal:is-located-in ?region.

  ?region support:has-pretty-name ?regionname.
  ?region portal:is-part-of ?country.

  ?country support:has-pretty-name ?countryname.
 
   FILTER regex(?regionname,"bayern","i")
}
ORDER by ?townname
LIMIT 7
"""
qres=endpoint.queryAsListOfDicts(query)
print(tabulate(qres))